In [2]:
import zipfile
from bs4 import BeautifulSoup
import os
import sys

In [30]:
d = zipfile.ZipFile("examples/train_examples/2.docx")

In [31]:
xml = d.read('word/document.xml')

In [36]:
bs = BeautifulSoup(xml, 'xml')

In [37]:
val = "ff2ad9"
color_bs = BeautifulSoup(f'<w:color w:val={val}/>').body.contents[0]

# if bs.body.p.pPr.rPr.color:
#     bs.body.p.pPr.rPr.color['w:val'] = val
# else:
#     new_tag = bs.new_tag('color')
#     bs.body.p.pPr.rPr.insert(1, new_tag)
#     bs.body.p.pPr.rPr.color['w:val'] = val
#     print(bs.body.p.pPr)
for r in bs.body.p:
#     if r.name == 'r':
    if r.rPr: 
        if r.rPr.color:
            r.rPr.color['w:val'] = val
        else:
            new_tag = bs.new_tag('color')
            r.rPr.insert(1, new_tag)
            r.rPr.color['w:val'] = val

In [38]:
bs.body.p

<w:p><w:pPr><w:pStyle w:val="Normal"/><w:rPr><w:b/><color w:val="ff2ad9"/><w:b/><w:bCs/><w:sz w:val="36"/><w:szCs w:val="36"/></w:rPr></w:pPr><w:r><w:rPr><w:b/><color w:val="ff2ad9"/><w:bCs/><w:sz w:val="36"/><w:szCs w:val="36"/></w:rPr><w:t>Пример документа</w:t></w:r></w:p>

In [39]:
namelist = d.namelist()
namelist

['_rels/.rels',
 'docProps/app.xml',
 'docProps/core.xml',
 'word/_rels/document.xml.rels',
 'word/settings.xml',
 'word/fontTable.xml',
 'word/document.xml',
 'word/styles.xml',
 '[Content_Types].xml']

In [40]:
d.extractall('examples/train_examples/tmp')

In [41]:
str(bs)

'<?xml version="1.0" encoding="utf-8"?>\n<w:document mc:Ignorable="w14 wp14" xmlns:mc="http://schemas.openxmlformats.org/markup-compatibility/2006" xmlns:o="urn:schemas-microsoft-com:office:office" xmlns:r="http://schemas.openxmlformats.org/officeDocument/2006/relationships" xmlns:v="urn:schemas-microsoft-com:vml" xmlns:w="http://schemas.openxmlformats.org/wordprocessingml/2006/main" xmlns:w10="urn:schemas-microsoft-com:office:word" xmlns:w14="http://schemas.microsoft.com/office/word/2010/wordml" xmlns:wp="http://schemas.openxmlformats.org/drawingml/2006/wordprocessingDrawing" xmlns:wp14="http://schemas.microsoft.com/office/word/2010/wordprocessingDrawing" xmlns:wpg="http://schemas.microsoft.com/office/word/2010/wordprocessingGroup" xmlns:wps="http://schemas.microsoft.com/office/word/2010/wordprocessingShape"><w:body><w:p><w:pPr><w:pStyle w:val="Normal"/><w:rPr><w:b/><color w:val="ff2ad9"/><w:b/><w:bCs/><w:sz w:val="36"/><w:szCs w:val="36"/></w:rPr></w:pPr><w:r><w:rPr><w:b/><color w:va

In [ ]:
with open('examples/train_examples/tmp/word/document.xml', 'w') as f:
    f.write(str(bs))

In [ ]:
d1 = zipfile.ZipFile("examples/train_examples/0.docx", mode='w')
for filename in namelist:
    d1.write(f'examples/train_examples/tmp/{filename}', arcname=filename)
d1.close()

In [ ]:
d2 = zipfile.ZipFile("examples/train_examples/0.docx", mode='r')

In [ ]:
d2.namelist()

In [ ]:
d = zipfile.ZipFile("examples/train_examples/0.docx")
xml = d.read('word/document.xml')
bs = BeautifulSoup(xml, 'xml')
bs